<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 07 &mdash; Working with Real-Time Data and Sockets**

## Visualizing Streaming Data with Plotly

### The Basics

In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'fe17a401-0e26-49de-8c3c-0a1d738a7c32'}], 'layout': {'template': '...'}
})

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

<SocketContext(connect='tcp://0.0.0.0:5555')>

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '65131e73-a9f4-45c5-838a-0466297e6109'},
             {'line': {'dash': 'dot', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA1',
              'type': 'scatter',
              'uid': '51947a1c-8715-4fab-ac39-9677ccb4ea00'},
             {'line': {'dash': 'dash', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA2',
              'type': 'scatter',
              'uid': '10cc9fc1-63c9-467b-8c89-c1fbc69866d0'}],
    'layout': {'template': '...'}
})

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '25224502-3a06-44b7-8400-43b57f140266',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'dash': 'dot', 'width': 1},
              'marker': {'symbol': 'triangle-up'},
              'mode': 'lines+markers',
              'name': 'RETURN',
              'type': 'scatter',
              'uid': '965a7d1b-3f86-43b8-941d-5db44be76272',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'dash': 'dash', 'width': 1},
              'marker': {'symbol': 'x'},
              'mode': 'lines+markers',
              'name': 'MOMENTUM',
              'type': 'scatter',
              'uid': '4e34f588-3445-4873-8537-1d4921ab0fd1',
              'xaxis': 'x3',
              'yaxis': 'y3'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
               'x

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = pd.concat((df, pd.DataFrame({sym: float(price)}, index=[t])))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [21]:
socket = context.socket(zmq.SUB)

In [22]:
socket.connect('tcp://0.0.0.0:5556')

<SocketContext(connect='tcp://0.0.0.0:5556')>

In [23]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [24]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

68.474 53.983 71.530 38.054 59.391 75.852 64.257 10.243
85.594 35.873 99.103 87.356 29.510 17.177 37.536 31.896
90.249 52.428 93.202 27.194 34.724 90.042 93.446 98.263
4.691 52.580 51.060 45.090 69.747 53.265 44.013 14.755
32.432 5.754 6.585 75.092 83.102 85.810 25.832 78.937


In [25]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': '2269afba-c4f6-4ce5-b905-6095f9a14e2a'}], 'layout': {'template': '...'}
})

In [26]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>